### 1. Import Libraries 

In [2]:
#Install pandas, numpy & requests
import pandas as pd
import numpy as np

#Install Beautiful Soup & Requests
!pip install beautifulsoup4
!pip install requests

from bs4 import BeautifulSoup
import requests

#Install libraries & packages
!conda install -c conda-forge folium=0.5.0 --yes
import folium

!conda install -c conda-forge geopy --yes

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



### 2. Scrape Data

In [7]:
#Collect HTML from Wiki page and create a soup object
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(source.text, 'lxml')

#Import the GeoData
geo_data=pd.read_csv("https://cocl.us/Geospatial_data")

### 3. Set Up Dataframe & Clean

In [8]:
#Set up table
data = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th', 'td']):
        section.append(td.text.rstrip())
    if (index == 0):
        columns = section
    else:
        data.append(section)
        
#Convert to dataframe
toronto_df1 = pd.DataFrame(data = data, columns = ['PostalCode', 'Borough', 'Neighbourhood'])

#Remove Boroughs that are 'Not Assigned'
toronto_df1.drop(toronto_df1[toronto_df1['Borough']=="Not assigned"].index, axis=0, inplace=True)

#Reindex df1 to df2
toronto_df2 = toronto_df1.reset_index()

#Multiple neighborhoods are listed under one postcode, therefore add groupby
toronto_df2 = toronto_df1.groupby("PostalCode").agg(lambda x:','.join(set(x)))

#Fix cells where the neighbourhood is missing, neighborhood = borough
toronto_df2.loc[toronto_df2['Neighbourhood']=="Not assigned",'Neighbourhood']=toronto_df2.loc[toronto_df2['Neighbourhood']=="Not assigned",'Borough']

#Reindex df2 to df3
toronto_df3 = toronto_df2.reset_index()

#Remove duplicate Borroughs 
toronto_df3['Borough']= toronto_df3['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")

#Join the data to the original dataframe and inspect details of updated dataframe
toronto_df3['Latitude'] = geo_data['Latitude'].values
toronto_df3['Longitude'] = geo_data['Longitude'].values
toronto_df3.info()
toronto_df3.shape
toronto_df3.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 5 columns):
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
Latitude         103 non-null float64
Longitude        103 non-null float64
dtypes: float64(2), object(3)
memory usage: 4.1+ KB


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Port Union,Highland Creek,Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Morningside,Guildwood,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### 4. Visualize the suburb data on a map

In [210]:
#Find center of Toronto 
c_lat = toronto_df3.Latitude.mean()
c_long = toronto_df3.Longitude.mean()

#Set map location/ variable
toronto_map = folium.Map(location = [c_lat, c_long], zoom_start = 12)

for lat, lng, borough, neighborhood in zip(toronto_df3['Latitude'], toronto_df3['Longitude'], toronto_df3['Borough'], toronto_df3['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7
        ).add_to(toronto_map) 


In [211]:
toronto_map

### 5. Define Foursquare Credentials

In [11]:
client_id = '0D2DKNC5DE5NMSKO15SIWBCYQD4UNPUUVCXGWNO1HBM5MNYG'
client_secret = 'KCF5OG12KFT2LOHALGUWPILYG3BJOQBB0TO2240PQH22NB2T'
version = '20180605'

### 6. Collect top nearby venues from Foursquare and amend the dataframe

In [195]:
#Set up getVenues from Foursquare
def getVenues(PostalCode, Latitude, Longitude, radius=500, LIMIT=100):
    
    venues_list = []
    for PostalCode, lat, lng in zip(PostalCode, Latitude, Longitude):
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            client_id, 
            client_secret, 
            version, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        results = requests.get(url).json()['response']['groups'][0]['items']
            
        venues_list.append([(
            PostalCode, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Postcode Latitude', 
                  'Postcode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
        
    return nearby_venues

In [196]:
#Set up Toronto Venues dataframe
toronto_venues = getVenues(PostalCode = toronto_df3['PostalCode'],
                                 Latitude = toronto_df3['Latitude'],
                                 Longitude = toronto_df3['Longitude'])

In [197]:
#First impressions of dataframe
toronto_venues.head()
toronto_venues.info()
toronto_venues.shape
toronto_venues.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2222 entries, 0 to 2221
Data columns (total 7 columns):
Postcode              2222 non-null object
Postcode Latitude     2222 non-null float64
Postcode Longitude    2222 non-null float64
Venue                 2222 non-null object
Venue Latitude        2222 non-null float64
Venue Longitude       2222 non-null float64
Venue Category        2222 non-null object
dtypes: float64(4), object(3)
memory usage: 121.6+ KB


,Postcode,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M1B,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,M1C,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,M1E,43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,M1E,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,M1E,43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


### 7. Set up a dataframe that shows top 5 venues per postcode

In [200]:
#Set up column data 
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Postcode'] = toronto_venues['Postcode'] 
cols = list(toronto_onehot)
cols.insert(0, cols.pop(cols.index('Postcode')))
toronto_onehot = toronto_onehot.loc[:, cols]

#Set up Toronto Grouped
toronto_grouped = toronto_onehot.groupby('Postcode').mean().reset_index()

#Write function to sort venues
def most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 5
indicators = ['st', 'nd', 'rd']

columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
postcodes_venues_sorted = pd.DataFrame(columns=columns)
postcodes_venues_sorted['Postcode'] = toronto_grouped['Postcode']

for ind in np.arange(toronto_grouped.shape[0]):
    postcodes_venues_sorted.iloc[ind, 1:] = most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

postcodes_venues_sorted.head()

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1B,Fast Food Restaurant,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run
1,M1C,Bar,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant
2,M1E,Medical Center,Pizza Place,Breakfast Spot,Mexican Restaurant,Electronics Store
3,M1G,Coffee Shop,Korean Restaurant,Yoga Studio,Electronics Store,Dog Run
4,M1H,Caribbean Restaurant,Bank,Hakka Restaurant,Fried Chicken Joint,Thai Restaurant


### 8. Analysing the Data

In [201]:
#Descriptive statistics on postcodes_venues_sorted
postcodes_venues_sorted.describe()

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
count,100,100,100,100,100,100
unique,100,43,51,50,63,53
top,M9N,Coffee Shop,Coffee Shop,Coffee Shop,Yoga Studio,Dog Run
freq,1,17,8,10,10,11


In [202]:
#List count of venues within region
venue_count = toronto_venues["Venue Category"].value_counts()
venue_count.head(10)

Coffee Shop           194
Café                   99
Restaurant             62
Park                   53
Pizza Place            50
Italian Restaurant     49
Bakery                 49
Bar                    42
Sandwich Place         41
Hotel                  41
Name: Venue Category, dtype: int64

In [147]:
#List of '1st Most Common Venues'
first_rank = postcodes_venues_sorted["1st Most Common Venue"].value_counts()
first_rank.head()

Coffee Shop             17
Café                     8
Park                     8
Fast Food Restaurant     5
Pizza Place              4
Name: 1st Most Common Venue, dtype: int64

In [135]:
#List of '2nd Most Common Venues'
second_rank = postcodes_venues_sorted["2nd Most Common Venue"].value_counts()
second_rank.head()

Coffee Shop    8
Park           8
Yoga Studio    7
Café           6
Bakery         3
Name: 2nd Most Common Venue, dtype: int64

In [136]:
#List of '3rd Most Common Venues'
third_rank = postcodes_venues_sorted["3rd Most Common Venue"].value_counts()
third_rank.head()

Coffee Shop                    10
Eastern European Restaurant     8
Yoga Studio                     7
Café                            5
Bakery                          4
Name: 3rd Most Common Venue, dtype: int64

In [137]:
#List of '4th Most Common Venues'
fourth_rank = postcodes_venues_sorted["4th Most Common Venue"].value_counts()
fourth_rank.head()

Yoga Studio          10
Discount Store        6
Dog Run               5
Electronics Store     3
Bar                   3
Name: 4th Most Common Venue, dtype: int64

In [141]:
#List of '5th Most Common Venues'
fifth_rank = postcodes_venues_sorted["5th Most Common Venue"].value_counts()
fifth_rank.head()

Dog Run               11
Italian Restaurant     6
Doner Restaurant       5
Sandwich Place         5
Diner                  4
Name: 5th Most Common Venue, dtype: int64

### 9. Observations

From this, we can see that 'Coffee Shops' are extremely popular, they rank first for '1st Most Common Venue', '2nd Most Common Venue' and '3rd Most Common Venue', however the market appears saturated with 194 'Coffe Shop' venues and 99 'Café' venues. 
'Yoga Studio' appears poppular, it is the first venue type following 'Coffee Shop' to rank first in common venue ranking. It ranks third for '2nd Most Common Venue', third for '3rd Most Common Venue' and first for '4th Most Common Venue'. 
While there are 194 Coffe Shops, there are only 10 Yoga Studios, placing it as one of the least occuring venue types. Therefore our analysis will look at the location of 'Yoga Studios' in Toronto and their geographical placement to idenitfy potential gaps in the market. We will start by generating a dataframe listing the Yoga Studios in Toronto, there should be 10. 

In [203]:
#Set up dataframe of just yoga studios
toronto_yoga_studios = toronto_venues.loc[toronto_venues['Venue Category'] == 'Yoga Studio']
toronto_yoga_studios

,Postcode,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
344,M4H,43.705369,-79.349372,Bikram Yoga East York,43.705450,-79.351448,Yoga Studio
366,M4K,43.679557,-79.352188,Moksha Yoga Danforth,43.677622,-79.352116,Yoga Studio
451,M4M,43.659526,-79.340923,Spirit Loft Yoga,43.663548,-79.341333,Yoga Studio
475,M4R,43.715383,-79.405678,Barreworks,43.714070,-79.400109,Yoga Studio
651,M4Y,43.665860,-79.383160,Bikram Yoga Yonge,43.668205,-79.385780,Yoga Studio
655,M4Y,43.665860,-79.383160,The Yoga Sanctuary,43.661499,-79.383636,Yoga Studio
1028,M5G,43.657952,-79.387383,The Yoga Sanctuary,43.661499,-79.383636,Yoga Studio
1936,M6J,43.647927,-79.419750,YogaSpace,43.647607,-79.420133,Yoga Studio
1996,M6K,43.636847,-79.428191,Pure Yoga Toronto,43.637330,-79.423800,Yoga Studio
2089,M7A,43.662301,-79.389494,The Yoga Sanctuary,43.661499,-79.383636,Yoga Studio


It appears that there are duplicates, with 'The Yoga Sanctuary' appearing three times, each record with the same latitude and longitude. Let's remove the duplicates.

In [204]:
#Remove duplicates from dataframe
toronto_yoga_studios = toronto_yoga_studios.drop_duplicates(subset='Venue', keep='first')
toronto_yoga_studios

,Postcode,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
344,M4H,43.705369,-79.349372,Bikram Yoga East York,43.705450,-79.351448,Yoga Studio
366,M4K,43.679557,-79.352188,Moksha Yoga Danforth,43.677622,-79.352116,Yoga Studio
451,M4M,43.659526,-79.340923,Spirit Loft Yoga,43.663548,-79.341333,Yoga Studio
475,M4R,43.715383,-79.405678,Barreworks,43.714070,-79.400109,Yoga Studio
651,M4Y,43.665860,-79.383160,Bikram Yoga Yonge,43.668205,-79.385780,Yoga Studio
655,M4Y,43.665860,-79.383160,The Yoga Sanctuary,43.661499,-79.383636,Yoga Studio
1936,M6J,43.647927,-79.419750,YogaSpace,43.647607,-79.420133,Yoga Studio
1996,M6K,43.636847,-79.428191,Pure Yoga Toronto,43.637330,-79.423800,Yoga Studio


In [212]:
#Set up map of Toronoto with location of yoga studios listed
c_lat = toronto_df3.Latitude.mean()
c_long = toronto_df3.Longitude.mean()

#Set map location/ variable
toronto_yoga_map = folium.Map(location = [c_lat, c_long], zoom_start = 13)

for lat, lng, venue in zip(toronto_yoga_studios['Venue Latitude'], toronto_yoga_studios['Venue Longitude'], toronto_yoga_studios['Venue']):
    label = '{}'.format(venue)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7
        ).add_to(toronto_yoga_map) 

In [213]:
toronto_yoga_map

Let us assume that proximity is a concern and that the radius can be used as a rough estimate to determine areas that may not have a Yoga Studio within a certain proximity.

In [208]:
#Setup a radius of the yoga studios at 100
toronto_yoga_map2 = folium.Map(location = [c_lat, c_long], zoom_start = 13)

for lat, lng, venue in zip(toronto_yoga_studios['Venue Latitude'], toronto_yoga_studios['Venue Longitude'], toronto_yoga_studios['Venue']):
    label = '{}'.format(venue)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 100,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7
        ).add_to(toronto_yoga_map2) 

In [191]:
toronto_yoga_map2

### Next Steps

From this we can see potential unserviced geographic regions within Toronto. There are gaps where the studio may not be close enough to service relevant market segments. Addtional analysis should look at;

1. What is the average distance travelled to and from a Yoga Studio by clientel. After obtaining this information, update the radius to better reflect unserviced geographic regions. 
2. What are key demographic features of Yoga Stuido clientle, demographics, age, location, socio-economic status etc. Once this data is collected, map it by regions and overlay it on the serviced region map. 
3. Shortlist 'Top 5 Unserviced Regions' based on points 1 and 2. Then conduct further analysis, identifying potential rental costs and locations, nearby features that may value add and traffic conditions servicing the region at peak times. 

With the above steps completed, narrow the list to the top 3 rental locations for a studio. 